# Imports

In [ ]:
!pip install geopandas
!pip install pyshp
!pip install shapely
!pip install geojson
!pip install ecmwf-api-client
!pip install eccodes
!pip install os
!pip install xarray[complete]
!pip install cfgrib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 24.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 974.5 kB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for ecmwf-api-client: filename=ecmwf_api_client-1.6.3-py3-none-any.whl size=13530 sha256=aacf2bc3d097ecf464410d387a94c670ae54223

In [ ]:
import os
import gc
from google.colab import drive
import shapefile
# import cfgrib

import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import shape
import pyproj
import geojson

import requests
import json

# Global Variables and Google Drive Connect

In [ ]:
USER = 'Tom'

In [ ]:
# Mount drive where you will do your work.
drive.mount('/content/drive')
if USER == 'Tom': 
  root_dir = "/content/drive/My Drive/W210/"
  project_folder = "Capstone/"
elif USER == 'Bri':
  pass
  # root_dir = "/content/drive/My Drive/266/"
  # project_folder = "Final Project/"
elif USER == 'Bronte':
  pass
  # root_dir = "/content/drive/My Drive/266/"
  # project_folder = "Final Project/"
elif USER == 'Kurt':
  pass
  # root_dir = "/content/drive/My Drive/266/"
  # project_folder = "Final Project/"
else:
  raise Exception("User unrecognized, must connect to shared drive")

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  # to test if all is working, you can uncomment these two lines below--it should write a file to the shared drive
  # !touch 'new_file_in_working_directory.txt'
# print('\nYour working directory was changed to ' + root_dir + project_folder + \
#         "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

os.chdir(root_dir + project_folder)

Mounted at /content/drive


# Load Data and Create Data Sets

In [ ]:
wdf = gpd.read_file("Data/Geo/Wind_Resource_Areas_(2022)/Wind_Resource_Areas_(2022).shp")
# Inspect the dataframe
print(wdf.head())
print(wdf.info())
print(wdf.columns)

   OBJECTID            Name    Shape__Are     Shape__Len  \
0         1       Tehachapi  4.296438e+09  257308.219066   
1         2        Altamont  3.098385e+09  269606.594561   
2         3          Solano  6.934007e+09  396223.812316   
3         4    San Gorgonio  8.137936e+09  377484.495992   
4         5  East San Diego  6.927814e+08  110717.037676   

                                            geometry  
0  POLYGON ((-13160874.615 4116568.498, -13161007...  
1  POLYGON ((-13490492.576 4332485.079, -13490627...  
2  POLYGON ((-13509186.741 4519476.490, -13509324...  
3  POLYGON ((-12973665.028 4001879.929, -12973796...  
4  POLYGON ((-12941722.714 3846550.974, -12941852...  
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    5 non-null      int64   
 1   Name        5 non-null      object  
 2   Shape__Are  5 non-null      

In [ ]:
sdf = gpd.read_file("Data/Geo/Solar_Resource_Areas_(2022)/Solar_Resource_Areas_(2022).shp")
# Inspect the dataframe
print(sdf.head())
print(sdf.info())
print(sdf.columns)

   OBJECTID                     Name    Shape__Are     Shape__Len  \
0         1                 Bay Area  3.466896e+10  835360.229135   
1         2                 Imperial  2.914968e+10  671051.820231   
2         3       Mid Central Valley  3.116761e+10  747868.356108   
3         4  Northern Central Valley  1.241350e+10  551117.710022   
4         5          San Luis Obispo  2.115097e+10  697840.063446   

                                            geometry  
0  POLYGON ((-13555295.035 4417731.934, -13556085...  
1  POLYGON ((-12873979.529 3844537.015, -12874797...  
2  POLYGON ((-13431888.302 4414837.101, -13432874...  
3  POLYGON ((-13524183.466 4728073.297, -13524907...  
4  POLYGON ((-13406049.190 4145615.205, -13406999...  
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    8 non-null      int64   
 1   Name        8 no

In [ ]:
def get_centroid_coordinates(shp_file):
    # Read the shapefile
    gdf = gpd.read_file(shp_file)
    
    # Calculate the centroid of each polygon
    gdf['centroid'] = gdf.centroid

    # Extract the latitude and longitude coordinates of each centroid
    names = gdf['Name']
    latitudes = gdf['centroid'].y
    longitudes = gdf['centroid'].x

    # Return the coordinates as a list of (latitude, longitude) tuples
    return list(zip(names, latitudes, longitudes))

# Call
solar_centroid_coordinates = get_centroid_coordinates("Data/Geo/Solar_Resource_Areas_(2022)/Solar_Resource_Areas_(2022).shp")
print(solar_centroid_coordinates)

wind_centroid_coordinates = get_centroid_coordinates("Data/Geo/Wind_Resource_Areas_(2022)/Wind_Resource_Areas_(2022).shp")
print(wind_centroid_coordinates)

[('Bay Area', 4596987.439929786, -13619226.143286793), ('Imperial', 3962299.909002147, -12877486.3638216), ('Mid Central Valley', 4563930.310944842, -13461838.83024754), ('Northern Central Valley', 4841803.237539923, -13575280.471651021), ('San Luis Obispo', 4311751.4661832815, -13479031.970143065), ('South Central Valley', 4290466.083882093, -13312393.392629797), ('Los Angeles Metro', 3961565.683265518, -13081691.305699667), ('Tehachapi', 4138330.8183525978, -13096704.130538512)]
[('Tehachapi', 4170516.747536039, -13161229.472480835), ('Altamont', 4387912.782929703, -13499463.576707425), ('Solano', 4593225.729188897, -13555066.756273514), ('San Gorgonio', 4055113.817660905, -13028383.022387657), ('East San Diego', 3858284.797278278, -12936540.971351812)]


In [ ]:
def shp_to_centroids(shp_file):
    # Load the shapefile into a GeoDataFrame
    gdf = gpd.read_file(shp_file)
    
    # Compute the centroid for each shape
    gdf['centroid'] = gdf['geometry'].centroid
    
    # Extract the shape name, latitude, and longitude of the centroid of each shape
    centroids = []
    for i, row in gdf.iterrows():
        name = row['Name']
        lat, lng = row['centroid'].y, row['centroid'].x
        centroids.append((name, lat, lng))
    
    return centroids

# Call
solar_centroid_coordinates = shp_to_centroids("Data/Geo/Solar_Resource_Areas_(2022)/Solar_Resource_Areas_(2022).shp")
print(solar_centroid_coordinates)

wind_centroid_coordinates = shp_to_centroids("Data/Geo/Wind_Resource_Areas_(2022)/Wind_Resource_Areas_(2022).shp")
print(wind_centroid_coordinates)

[('Bay Area', 4596987.439929786, -13619226.143286793), ('Imperial', 3962299.909002147, -12877486.3638216), ('Mid Central Valley', 4563930.310944842, -13461838.83024754), ('Northern Central Valley', 4841803.237539923, -13575280.471651021), ('San Luis Obispo', 4311751.4661832815, -13479031.970143065), ('South Central Valley', 4290466.083882093, -13312393.392629797), ('Los Angeles Metro', 3961565.683265518, -13081691.305699667), ('Tehachapi', 4138330.8183525978, -13096704.130538512)]
[('Tehachapi', 4170516.747536039, -13161229.472480835), ('Altamont', 4387912.782929703, -13499463.576707425), ('Solano', 4593225.729188897, -13555066.756273514), ('San Gorgonio', 4055113.817660905, -13028383.022387657), ('East San Diego', 3858284.797278278, -12936540.971351812)]


In [ ]:
def geoJSON_to_centroids(geoJSON_file):
    # Load the GeoJSON file into a GeoDataFrame
    gdf = gpd.read_file(geoJSON_file)
    
    # Compute the centroid for each shape
    gdf['centroid'] = gdf['geometry'].centroid
    
    # Extract the shape name, latitude, and longitude of the centroid of each shape
    centroids = []
    for i, row in gdf.iterrows():
        name = row['Name']
        lat, lng = row['centroid'].y, row['centroid'].x
        centroids.append((name, lat, lng))
    
    return centroids

# Call
solar_centroid_coordinates = geoJSON_to_centroids("Data/Geo/Solar_Resource_Areas_(2022)/Solar_Resource_Areas_(2022).geojson")
print(solar_centroid_coordinates)
df_solar = pd.DataFrame(solar_centroid_coordinates, columns=["Name", "Latitude", "Longitude"])
# display(df_solar)

wind_centroid_coordinates = geoJSON_to_centroids("Data/Geo/Wind_Resource_Areas_(2022)/Wind_Resource_Areas_(2022).geojson")
print(wind_centroid_coordinates)
df_wind = pd.DataFrame(wind_centroid_coordinates, columns=["Name", "Latitude", "Longitude"])
display(df_wind)

<ipython-input-10-a7533e531bf4>:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf['geometry'].centroid


[('Bay Area', 38.11695842695488, -122.3417829400716), ('Imperial', 33.50076843810364, -115.67988661615553), ('Mid Central Valley', 37.884245921907585, -120.92871618700734), ('Northern Central Valley', 39.82977582914325, -121.94780613519598), ('San Luis Obispo', 36.07619766133289, -121.08308314245073), ('South Central Valley', 35.920664650332036, -119.58624401463064), ('Los Angeles Metro', 33.49344998398914, -117.51403652562306), ('Tehachapi', 34.810745603790934, -117.64915969169141)]
[('Tehachapi', 35.04821599273374, -118.22938262414684), ('Altamont', 36.630901466054205, -121.26775068802537), ('Solano', 38.095790755129464, -121.76704449041097), ('San Gorgonio', 34.19489791995151, -117.03582187948841), ('East San Diego', 32.72063526491512, -116.2109382686741)]


<ipython-input-10-a7533e531bf4>:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf['geometry'].centroid


,Name,Latitude,Longitude
0,Tehachapi,35.048216,-118.229383
1,Altamont,36.630901,-121.267751
2,Solano,38.095791,-121.767044
3,San Gorgonio,34.194898,-117.035822
4,East San Diego,32.720635,-116.210938


In [ ]:
solar_wind_area_names = []
for swc in solar_centroid_coordinates:
  solar_wind_area_names.append(swc[0].replace(" ","_"))
print(f'Solar_Wind_Area_Names: {solar_wind_area_names}')

Solar_Wind_Area_Names: ['Bay_Area', 'Imperial', 'Mid_Central_Valley', 'Northern_Central_Valley', 'San_Luis_Obispo', 'South_Central_Valley', 'Los_Angeles_Metro', 'Tehachapi']
